In [1]:
import numpy as np
import matplotlib.pyplot as plt
import sys
sys.path.append("C:/Users/82103/Py_3_10/deep-learning-from-scratch")
from common.functions import softmax, cross_entropy_error

In [2]:
# 미분 함수 구현

def numerical_diff(f, x):
    h = 1e-4
    return (f(x + h) - f(x - h)) / (2*h)

# 변수가 2개인 함수
def function_2(x):
    return np.sum(x**2)


In [3]:
# 다변수 함수에 대한 기울기 값 계산 -> 각 변수에 대한 편미분 값을 계산 후 벡터로 표현
def _numerical_gradient_no_batch(f, x):
    h = 1e-4
    grad = np.zeros_like(x)
    for idx in range(x.size):
        tmp_val = x[idx]
        
        # f(x + h)의 함수값 계산 (idx번째 변수x + h)
        x[idx] = tmp_val + h
        fxh1 = f(x)
        
        x[idx] = tmp_val - h
        fxh2 = f(x)
        
        grad[idx] = (fxh1 - fxh2) / (2*h)
        x[idx] = tmp_val
        
    return grad


In [4]:
# 입력 X의 차원에 따른 구분을 해준 기울기 구하는 함수
def numerical_gradient(f, X):
    # X가 1차원일 경우
    if X.ndim == 1:
        return _numerical_gradient_no_batch(f, X)
    # X가 다차원일 경우
    else:
        grad = np.zeros_like(X)
        
        for idx, x in enumerate(X):
            grad[idx] = _numerical_gradient_no_batch(f, x)
        
        return grad

In [5]:
def gradient_descent(f, init_x, lr=0.01, step_num=100):
    x = init_x
    
    for i in range(step_num):
        grad = numerical_gradient(f, x)
        x -= lr * grad
        
    return x

In [6]:
# function_2(x)에 대해 최소가 되는 x값을 찾고 최솟값을 구하자
# 시작 x값 설정 (x0, x1)
init_x = np.array([-3.0, 4.0])
x = gradient_descent(function_2, init_x, 0.1, 100)
print(x)
y = function_2(x)
print(y)

# lr값을 어떻게 설정하느냐에 따라 최소가 되는 값이 매우 다양하게 바뀜
# -> 적당한 lr값 설정하는 것도 중요

[-6.11110793e-10  8.14814391e-10]
1.0373788922158197e-18


In [7]:
class simpleNet:
    def __init__(self):
        # 가중치 값 정규분포로 임의로 초기화 (2x3 행렬형태)
        self.W = np.random.randn(2, 3)
    
    def predict(self, x):
        return np.dot(x, self.W)
    
    # 임의의 가중치에 대한 출력에 대한 손실함수까지 계산해서 반환
    def loss(self, x, t):
        # 단층 퍼셉트론 형태의 신경망 구현(은닉층 존재 x)
        z = self.predict(x)
        # 활성화 함수로 softmax함수 사용 -> 사용효과 : 출력값이 확률의 의미를 가짐
        y = softmax(z)
        loss = cross_entropy_error(y, t)
        
        return loss

In [8]:
net = simpleNet()
print(net.W)

[[ 1.43330571  1.42540611 -0.11877021]
 [-1.65888174  0.43329911 -0.02130455]]


In [9]:
x = np.array([0.6, 0.9])
p = net.predict(x)
print(p)
print(np.argmax(p))
# 정답 레이블
t = np.array([0, 0, 1])
# 임의의 가중치에 대한 출력
net.loss(x, t)

[-0.63301014  1.24521287 -0.09043623]
1


1.683377849019081

In [10]:
def f(W):
    return net.loss(x, t)

# 각 가중치에 대한 편미분값을 나타낸 기울기 dW 
dW = numerical_gradient(f, net.W)
print(dW)

[[ 0.0647787   0.4237738  -0.4885525 ]
 [ 0.09716805  0.6356607  -0.73282875]]


In [11]:
W = gradient_descent(f, net.W, 0.01, 100)